In [1]:
import re
import pandas as pd
from collections import Counter
from konlpy.tag import Okt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
#민원 목록
minwonlist= pd.read_csv("minwonlist.csv", index_col=0)
minwonlist=minwonlist.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
##삭제 민원 제거
a_rows = minwonlist[minwonlist['title'] == '[관리자에 의해 삭제되었습니다.]']
minwonlist = minwonlist.drop(a_rows.index)

In [ ]:
#민원 내용
minwon1= pd.read_csv("minwon31.csv", index_col=0)
minwon2=pd.read_csv("minwon32.csv", index_col=0)
minwon3= pd.read_csv("minwon3251.csv", index_col=0)
minwon4= pd.read_csv("minwon5275.csv", index_col=0)
minwon5= pd.read_csv("minwon77.csv", index_col=0)
minwoncontents = pd.concat([minwon1,minwon2,minwon3,minwon4,minwon5])
minwoncontents = minwoncontents.drop('page', axis=1)
minwoncontents=minwoncontents.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
minwoncontents

In [ ]:
merged_df = pd.merge(minwonlist, minwoncontents, on='title')

In [ ]:
#답변완료된 df
##다부처병렬,이송이첩 상태 민원 제거
merged_df=merged_df[merged_df['answer_x']=='답변완료']
merged_df

In [2]:
minwondata= pd.read_csv("minwon_df.csv")
minwondata.head()

,title,date,answer_x,content,answer_y,depart
0,주차장이 너무 어두워요,2020.11.30,답변완료,중구 전동 25-328 번지옆 전동쉼터 옆 공영주차장이 야간에 너무 어두워요\n...,1. 구정 발전에 관심을 가져주시는 귀하께 감사의 말씀을 드립니다.2. 민원 제기하...,도시재생국 교통운수과
1,뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요?,2020.11.29,답변완료,아니 거기 오는 사람들을 위해서 버스정류장을 만들고 해안 데크길을 설치 했으면 그런...,1. 안녕하십니까? 귀하께서 상담민원을 통해 신청하신 민원에 대한 검토 결과를 다음...,국제도시건설국 농수산과
2,9100번 광역버스 배차간격 준수 요청 건,2020.11.23,답변완료,집안 사정으로 서구에서 남동구를 들려서 9100번 버스를 통해 인천에서 서울로 출퇴...,불편사항이 해결됨,도시재생국 교통운수과
3,46번버스 신흥교통 정상이아닌 버스업체,2020.11.23,답변완료,46번버스 신흥교통 손님에게 막말하고 잘못하고도 사과전혀없고 그냥 배째라는식으로 직...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시재생국 교통운수과
4,살려주세요 제발 요즘 가게 망하게 만드는 일들만 일어나네요..제발요(영종도 호텔),2020.11.19,답변완료,요즘 전부 어렵죠?\n영종도는 공항 상대로 영업을 하는 가게들이 많아서 특히나 더 ...,1. 구정 발전을 위해 관심을 가져주시는 귀하께 깊은 감사를 드립니다.2. 모두가 ...,국제도시건설국 도시공원과


In [3]:
minwondata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1227 entries, 0 to 1226
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     1227 non-null   object
 1   date      1227 non-null   object
 2   answer_x  1227 non-null   object
 3   content   1227 non-null   object
 4   answer_y  1226 non-null   object
 5   depart    1227 non-null   object
dtypes: object(6)
memory usage: 57.6+ KB


## 담당 부서별 민원 확인

In [4]:
#문장부호 제거
##온점은 남기기, 뛰어쓰기 두개 이상은 하나로
def remove_punctuation(text):
    """Remove punctuation from text"""
    #clean_text = re.sub('[^ㄱ-ㅣ가-힣0-9a-zA-Z]+', ' ', text)
    text=re.sub(r'[^\w\s.]', '', text)
    text=text.replace('\n', '')
    text=re.sub('\s+', ' ', text)
    return text

minwondata['content'] = minwondata['content'].apply(remove_punctuation)

In [5]:
minwondata['depart'].value_counts()

영종용유지원단 친환경조성과    163
도시재생국 건축과          48
도시관리국 건설과          43
지역개발국 교통운영과        41
도시관리국 교통운수과        38
                 ... 
행정복지국 복지과           1
국제도시행정국 여성보육과       1
국제도시행정국 도시행정과       1
행정복지국 세무1과          1
관광지원국 건설재난관리과       1
Name: depart, Length: 111, dtype: int64

In [6]:
#영종용유지원단 친환경조성과 관련 민원 삭제
minwondata = minwondata[~minwondata['depart'].str.contains('영종')]
"""minwondata = minwondata[minwondata['depart'] != '영종용유지원단 친환경조성과']
minwondata = minwondata[minwondata['depart'] != '영종출장소']
minwondata = minwondata[minwondata['depart'] != '영종용유지원단 기반시설과']"""
# 2014년 전 민원존재
minwondata = minwondata[~minwondata['depart'].str.contains('지역개발국')]

#주민생활지원국 가정교육과 삭제 -> 토큰수 큼
minwondata = minwondata[minwondata['depart'] != '주민생활지원국 가정교육과']

#변경된 부서이름 변경(아마두?)
minwondata['depart'] = minwondata['depart'].replace(
    {'도시관리국 건설과': '도시개발국 건설과',
     '도시재생국 건설과': '도시개발국 건설과',
     '도시관리국 건축과': '도시개발국 건축과',
     '도시재생국 건축과': '도시개발국 건축과',
     '도시재생국 환경보호과':'도시개발국 환경보호과',
     '도시관리국 교통운수과':'도시개발국 교통운수과',
     '도시재생국 교통운수과':'도시개발국 교통운수과'})

# 부서별 행의 수 계산
counts = minwondata['depart'].value_counts()

# 해당 부서별 행의 수가 10개 이상인 행만 선택하여 새로운 데이터프레임 생성
minwondata = minwondata[minwondata['depart'].isin(counts[counts >= 10].index)]
minwondata.head()


,title,date,answer_x,content,answer_y,depart
0,주차장이 너무 어두워요,2020.11.30,답변완료,중구 전동 25328 번지옆 전동쉼터 옆 공영주차장이 야간에 너무 어두워요야간에는 ...,1. 구정 발전에 관심을 가져주시는 귀하께 감사의 말씀을 드립니다.2. 민원 제기하...,도시개발국 교통운수과
1,뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요?,2020.11.29,답변완료,아니 거기 오는 사람들을 위해서 버스정류장을 만들고 해안 데크길을 설치 했으면 그런...,1. 안녕하십니까? 귀하께서 상담민원을 통해 신청하신 민원에 대한 검토 결과를 다음...,국제도시건설국 농수산과
2,9100번 광역버스 배차간격 준수 요청 건,2020.11.23,답변완료,집안 사정으로 서구에서 남동구를 들려서 9100번 버스를 통해 인천에서 서울로 출퇴...,불편사항이 해결됨,도시개발국 교통운수과
3,46번버스 신흥교통 정상이아닌 버스업체,2020.11.23,답변완료,46번버스 신흥교통 손님에게 막말하고 잘못하고도 사과전혀없고 그냥 배째라는식으로 직...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시개발국 교통운수과
4,살려주세요 제발 요즘 가게 망하게 만드는 일들만 일어나네요..제발요(영종도 호텔),2020.11.19,답변완료,요즘 전부 어렵죠영종도는 공항 상대로 영업을 하는 가게들이 많아서 특히나 더 어렵습...,1. 구정 발전을 위해 관심을 가져주시는 귀하께 깊은 감사를 드립니다.2. 모두가 ...,국제도시건설국 도시공원과


In [7]:
minwondata['depart'].value_counts()

도시개발국 교통운수과       73
도시개발국 건축과         68
도시개발국 건설과         64
국제도시건설국 기반시설과     29
국제도시건설국 도시공원과     23
기획감사실             18
국제도시건설국 교통과       18
도시개발국 환경보호과       18
국제도시행정국 친환경조성과    17
관광진흥실             17
보건소 보건행정과         16
주민생활지원국 청소과       15
홍보체육진흥실           14
주민생활지원국 환경관리과     13
주민생활지원국 자원순환과     13
도시관리국 교통행정과       13
용유출장소             13
보건소 건강증진과         13
국제도시국 교통지적과       13
국제도시건설국 농수산과      13
국제도시국 기반시설과       12
도시재생국 도시개발과       12
운서동               11
주민생활지원국 위생과       11
Name: depart, dtype: int64

## 임베딩 형태만들기

#### minwonDF <= title, content(+title), depart

In [9]:
minwon=pd.DataFrame()
minwon['title']=minwondata['title']

In [10]:
# content에 title넣어줌.
def combine_columns(row):
    return '민원 내용:'+str(row['title'])+str(row['content'])

# apply 함수를 사용하여 함수 적용
minwon['content'] = minwondata.apply(combine_columns, axis=1)

#민원 답변 추가
minwon['answer']=minwondata['answer_y']

#부서 추가
minwon['depart']=minwondata['depart']

In [12]:
stop_words = [
    '제발','제발요', 'ㅠ', '아니','너무','또한','게다가','너무','완전','그리고','굉장히','또','다시','도대체','자주','많이','몇일전','어제',
    '를','합니다','혹시','특히','더','안녕하세요','바로','습니','살려주세요'
            ] # 삭제하고 싶은 단어 리스트

for i, row in minwon.iterrows():
    content = row['content']
    for word in stop_words:
        content = content.replace(word, "")
    minwon.at[i, 'content'] = content


In [13]:
#중복제거
minwon.drop_duplicates(subset=['content'], inplace=True)

In [219]:
for i, row in minwon.iterrows():
    if '첨부파일' in row['content']:
        minwon.at[i, 'content'] = re.sub(r'첨부파일.*', '', row['content'])


In [220]:
minwon.head()

,title,content,answer,depart
0,주차장이 너무 어두워요,민원 내용:주차장이 어두워요중구 전동 25328 번지옆 전동쉼터 옆 공영주차장이 ...,1. 구정 발전에 관심을 가져주시는 귀하께 감사의 말씀을 드립니다.2. 민원 제기하...,도시개발국 교통운수과
1,뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요?,민원 내용:뭐에요? 거잠포선착장 쓰레기 그대로잖아요? 하나도 안 되어있네요? 거기 ...,1. 안녕하십니까? 귀하께서 상담민원을 통해 신청하신 민원에 대한 검토 결과를 다음...,국제도시건설국 농수산과
2,9100번 광역버스 배차간격 준수 요청 건,민원 내용:9100번 광역버스 배차간격 준수 요청 건집안 사정으로 서구에서 남동구 ...,불편사항이 해결됨,도시개발국 교통운수과
3,46번버스 신흥교통 정상이아닌 버스업체,민원 내용:46번버스 신흥교통 정상이아닌 버스업체46번버스 신흥교통 손님에게 막말하...,1. 교통 업무에 많은 관심과 애정을 가져주셔서 깊이 감사드립니다. 2. 귀하께서 ...,도시개발국 교통운수과
4,살려주세요 제발 요즘 가게 망하게 만드는 일들만 일어나네요..제발요(영종도 호텔),민원 내용: 요즘 가게 망하게 만드는 일들만 일어나네요..요(영종도 호텔)요즘 전...,1. 구정 발전을 위해 관심을 가져주시는 귀하께 깊은 감사를 드립니다.2. 모두가 ...,국제도시건설국 도시공원과
...,...,...,...,...
1171,을왕리해수욕장 식당호객행위 및 주차장 장악,민원 내용:을왕리해수욕장 식당호객행위 및 주차장 장악결혼기념일을 맞아 주말 3월2일...,"1. 먼저, 모처럼의 나들이에 을왕리해수욕장 내 호객행위로 인해 심려를 끼쳐 드린 ...",용유출장소
1184,영마루공원 풋살경기장 이용료 받아 소음방지시설 갖춰 주세요.,민원 내용:영마루공원 풋살경기장 이용료 받아 소음방지시설 갖춰 주세요.항상 구민 복...,새올 전자민원창구 민원상담 접수에 따라 붙임과 같이 답변하고자 합니다.붙 임 1. ...,홍보체육진흥실
1197,관내 보건소 명단 요청합니다.,민원 내용:관내 보건소 명단 요청.치과기공소 현황 문의 드립니다.의료관련 기관을 검...,"○ 저희 보건행정 업무에 관심 가져 주신 점 감사드리며, 귀하께서 제기한 민원에 대...",보건소 건강증진과
1199,교통사고 위험지역,민원 내용:교통사고 위험지역덕교동에서 공항전망대 방향으로 가는길에 커브길이 많아 사...,"귀하의 가정에 평화와 행복이 가득하시길 기원드리며, 건의하신 민원에 대해 답변드리겠...",용유출장소


In [221]:
minwon.to_csv("minwon.csv", encoding='utf-8-sig',mode='w',index=False)

----------